In [1]:
import os
import clip
import torch

In [2]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from tqdm import tqdm

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32",device)

In [5]:
root = os.path.expanduser("~/.cache")
train = CIFAR100(root, download=True, train=True ,transform=preprocess)
test = CIFAR100(root, download=True, train=False, transform=preprocess)

Extracting /home/jlew/.cache/cifar-100-python.tar.gz to /home/jlew/.cache
Files already downloaded and verified


In [6]:
def get_features(dataset):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()


In [7]:
# Calculate the image features
train_features, train_labels = get_features(train)
test_features, test_labels = get_features(test)

 34%|███▍      | 172/500 [08:28<16:06,  2.95s/it]

100%|██████████| 100/100 [04:54<00:00,  2.95s/it]


In [8]:
train_labels

array([19, 29,  0, ...,  3,  7, 73])

In [9]:
# Perform logistic regression
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/jlew/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.1min finished


LogisticRegression(C=0.316, max_iter=1000, random_state=0, verbose=1)

In [12]:
# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

Accuracy = 80.050
